# 0_2 ports clokmon
We exctract the C_sigma of a floating transmon qubit with two couplers by using QuCAT

In [73]:
from qucat import Network,L,J,C,R
import numpy as np
import pandas as pd

In [74]:
file_path = 'clockmon_capacitance_0_2_q3d_results.csv'
df = pd.read_csv(file_path)

In [75]:
Cmat = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        Cmat[i, j] = df["C"+str(i+1)+str(j+1)].values[0]

In [76]:
network_list = []
for i in range(0, 4):
    for j in range(i, 4):
        if i == j:
            network_list.append(C(0, j+1, Cmat[i, j]))
        else:
            network_list.append(C(i+1, j+1, Cmat[i, j]))
network_list.append(J(3, 4, 'Lj'))

In [77]:
cir = Network(network_list)
Lj = 10e-9
f,k,A,chi = cir.f_k_A_chi(pretty_print=True,  Lj = Lj)
ham = cir.hamiltonian(modes = [0],
                    taylor = 12,
                    excitations = [10],
                    Lj = Lj)
# ee = ham.eigenenergies()
# ee -= ee[0]
c_sigma_extracted = 1/Lj/(2*np.pi*f[0])**2
print("C_sigma extracted = ", c_sigma_extracted, "fF")

         mode |       freq.  |       diss.  |       anha.  |
            0 |     5.34 GHz |          0Hz |      218 MHz |

Kerr coefficients (diagonal = Kerr, off-diagonal = cross-Kerr)
         mode |         0    |
            0 |      218 MHz |

C_sigma extracted =  8.883685926355346e-14 fF


# Let's consider a coupler second mode

In [78]:
network_list = []
for i in range(0, 4):
    for j in range(i, 4):
        if i == j:
            network_list.append(C(0, j+1, Cmat[i, j]))
        else:
            network_list.append(C(i+1, j+1, Cmat[i, j]))
network_list.append(J(3, 4, 'Lj'))


planck_h = 6.62607015e-34
e_charge = 1.60217663e-19
phi0 =  2.067833848e-15 # Wb

a = 10e-6
b = 6e-6
ep_r = 11.45

rr_freq_target = 7e9
Z0 = 50
l_r = Z0*np.pi/(4*np.pi*rr_freq_target)
c_r = 1/((2*np.pi*rr_freq_target)**2*l_r)
network_list.append(C(0, 2, c_r))
network_list.append(L(0, 2, l_r))


In [79]:
cir = Network(network_list)
Lj = 14e-9
f,k,A,chi = cir.f_k_A_chi(pretty_print=True,  Lj = Lj)
ham = cir.hamiltonian(modes = [0, 1],
                    taylor = 12,
                    excitations = [10, 10],
                    Lj = Lj)
ee = ham.eigenenergies()
ee -= ee[0]
c_sigma_extracted = 1/Lj/(2*np.pi*f[0])**2
print("C_sigma extracted = ", c_sigma_extracted, "fF")

         mode |       freq.  |       diss.  |       anha.  |
            0 |     4.51 GHz |          0Hz |      217 MHz |
            1 |     6.42 GHz |          0Hz |       200 Hz |

Kerr coefficients (diagonal = Kerr, off-diagonal = cross-Kerr)
         mode |         0    |         1    |
            0 |      217 MHz |              |
            1 |      417 kHz |       200 Hz |

C_sigma extracted =  8.906189136775308e-14 fF


In [80]:
ee[1].real

4277690888.5638857

In [81]:
c_qr = np.abs((Cmat[1, 2]* Cmat[3,3] - Cmat[1, 3]*Cmat[2,2]) / (Cmat[2,2] + Cmat[1,2] + Cmat[3,3] + Cmat[3,1]))
EC = e_charge**2/(c_sigma_extracted)/2/planck_h


Ic = phi0/2/np.pi/Lj
Ej = Ic*phi0/2/np.pi/planck_h

In [82]:
g = 2*np.pi*EC/ (e_charge) * (Ej/EC/2)**(1/4) * c_qr / (c_r) * np.sqrt(2*planck_h * rr_freq_target * (c_r))

In [83]:
omega_r = rr_freq_target*2*np.pi
omega_q1 = ee[1].real*2*np.pi
omega_q2 = (ee[1].real + 0.1e9)*2*np.pi
J = 0.5 * g**2*(omega_q1 + omega_q2 - 2*omega_r) / (omega_q1 - omega_r) / (omega_q2 - omega_r)

In [84]:
J

-7170653.759962644